Thanks [ahsuna123](https://www.kaggle.com/code/ahsuna123/hull-tactic-no-overfitting-leakage) and [morodertobias](https://www.kaggle.com/code/morodertobias/hull-leak-safe-baseline) for sharing

感谢[ahsuna123](https://www.kaggle.com/code/ahsuna123/hull-tactic-no-overfitting-leakage)和[morodertobias](https://www.kaggle.com/code/morodertobias/hull-leak-safe-baseline)的分享

Remove the last 180 data points of train.csv from the training set to make the scores on the leaderboard meaningful

将train.csv的最后180条数据从训练集中删除，以使得leaderboard上的分数具备实际意义

Predicting positions directly from features

直接从特征中预测仓位

If features such as 3-day rolling average are manually added, it will cause the score given by the rating function called by the code to be inconsistent with the submitted result, because after submission, the data is passed in item by item, and features cannot be extracted based on adjacent data

如果手动添加例如3日滚动平均之类的特征，会导致代码调用的评分函数给出的分数与提交结果不一致，因为提交后，数据是逐条传入的，无法基于相邻数据提取特征

In [ ]:
import numpy as np
import pandas as pd
import pandas.api.types

MIN_INVESTMENT = 0
MAX_INVESTMENT = 2


class ParticipantVisibleError(Exception):
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """
    Calculates a custom evaluation metric (volatility-adjusted Sharpe ratio).

    This metric penalizes strategies that take on significantly more volatility
    than the underlying market.

    Returns:
        float: The calculated adjusted Sharpe ratio.
    """

    if not pandas.api.types.is_numeric_dtype(submission['prediction']):
        raise ParticipantVisibleError('Predictions must be numeric')

    solution = solution
    solution['position'] = submission['prediction']

    if solution['position'].max() > MAX_INVESTMENT:
        raise ParticipantVisibleError(f'Position of {solution["position"].max()} exceeds maximum of {MAX_INVESTMENT}')
    if solution['position'].min() < MIN_INVESTMENT:
        raise ParticipantVisibleError(f'Position of {solution["position"].min()} below minimum of {MIN_INVESTMENT}')

    solution['strategy_returns'] = solution['risk_free_rate'] * (1 - solution['position']) + solution['position'] * solution['forward_returns']

    # Calculate strategy's Sharpe ratio
    strategy_excess_returns = solution['strategy_returns'] - solution['risk_free_rate']
    strategy_excess_cumulative = (1 + strategy_excess_returns).prod()
    strategy_mean_excess_return = (strategy_excess_cumulative) ** (1 / len(solution)) - 1
    strategy_std = solution['strategy_returns'].std()

    trading_days_per_yr = 252
    if strategy_std == 0:
        raise ParticipantVisibleError('Division by zero, strategy std is zero')
    sharpe = strategy_mean_excess_return / strategy_std * np.sqrt(trading_days_per_yr)
    strategy_volatility = float(strategy_std * np.sqrt(trading_days_per_yr) * 100)

    # Calculate market return and volatility
    market_excess_returns = solution['forward_returns'] - solution['risk_free_rate']
    market_excess_cumulative = (1 + market_excess_returns).prod()
    market_mean_excess_return = (market_excess_cumulative) ** (1 / len(solution)) - 1
    market_std = solution['forward_returns'].std()

    market_volatility = float(market_std * np.sqrt(trading_days_per_yr) * 100)

    if market_volatility == 0:
        raise ParticipantVisibleError('Division by zero, market std is zero')

    # Calculate the volatility penalty
    excess_vol = max(0, strategy_volatility / market_volatility - 1.2) if market_volatility > 0 else 0
    vol_penalty = 1 + excess_vol

    # Calculate the return penalty
    return_gap = max(
        0,
        (market_mean_excess_return - strategy_mean_excess_return) * 100 * trading_days_per_yr,
    )
    return_penalty = 1 + (return_gap**2) / 100

    # Adjust the Sharpe ratio by the volatility and return penalty
    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    return min(float(adjusted_sharpe), 1_000_000)

In [ ]:
import os
import pandas as pd
import polars as pl
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.model_selection import TimeSeriesSplit
import lightgbm as lgb
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
import kaggle_evaluation.default_inference_server
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

class FixedSP500Pipeline:
    """Fixed end-to-end pipeline for S&P 500 returns prediction"""

    def __init__(self, feature_selection_k=50, manual_features=None):
        """
        Args:
            feature_selection_k: 自动特征选择时保留的特征数量
            manual_features: 手动指定特征名列表。如果传入此参数，则直接使用这些特征而非自动筛选。
        """
        self.feature_selection_k = feature_selection_k
        self.manual_features = manual_features

        self.scaler = RobustScaler()
        self.feature_selector = SelectKBest(
            lambda X, y: mutual_info_regression(X, y, random_state=42),
            k=feature_selection_k
        )

        self.models = {
            'xgb': xgb.XGBRegressor(
                n_estimators=100,
                learning_rate=0.01,
                verbosity=0,
                random_state=42,
            ),
        }

        self.model_weights = {'xgb': 1}
        self.feature_names = None
        self.all_feature_names = None
        # 用于保存最初（未经特征工程）的原始特征名称
        self.original_features = None  # Store original feature list
        self.is_fitted = False

    def create_features(self, df_batch):
        """Create additional features - CONSISTENT between training and prediction"""
        # df_batch：传入的一批数据（可以是整个训练集或单条测试数据）
        df_features = df_batch.copy()

        # Get base feature columns
        # 过滤出基础特征列（排除日期、目标等）
        feature_cols = [col for col in df_batch.columns
                        if not col.startswith(('date_id', 'forward_returns',
                                               'risk_free_rate', 'market_forward_excess_returns',
                                               'is_scored', 'lagged_'))]
        # 用于后续预测阶段就能检查特征是否一致
        if self.original_features is None:
            self.original_features = feature_cols

        # Simple interaction
        # 在金融建模中：E1 通常代表 Earnings（盈利类指标）；M1 通常代表 Market factor（市场动量或市值类指标）。
        # 两者相乘可以反映：“当盈利强劲且市场动量也高” 的共振效应，即市场中强者恒强的复合信号。
        # 交互项能够让模型学习特征间的非线性关系：例如：当 E1 很高但 M1 很低 → 也许不是好信号；当两者都高 → 未来回报率可能更高。
        if 'E1' in df_features.columns and 'M1' in df_features.columns:
            df_features['E1_M1_interaction'] = df_features['E1'] * df_features['M1']

        df_features["U1"] = df_features["I2"] - df_features["I1"]
        df_features["U2"] = df_features["M11"] / ((df_features["I2"] + df_features["I9"] + df_features["I7"]) / 3)

        return df_features

    def prepare_features(self, df, is_training=False):
        """Prepare features for modeling - CONSISTENT approach"""
        # is_training: 是否为训练阶段（布尔值）；True → 会 fit scaler（标准化器）；False → 只 transform（保证一致性）

        # 调用上一层的特征生成
        df_enhanced = self.create_features(df)

        feature_cols = [col for col in df_enhanced.columns
                        if not col.startswith(('date_id', 'forward_returns',
                                               'risk_free_rate', 'market_forward_excess_returns',
                                               'is_scored', 'lagged_'))]

        X = df_enhanced[feature_cols].copy()

        # Clip outliers异常值截断。所有小于 1% 分位数的值 → 设为 q01；所有大于 99% 分位数的值 → 设为 q99。
        for col in X.columns:
            q99 = X[col].quantile(0.99)
            q01 = X[col].quantile(0.01)
            X[col] = X[col].clip(q01, q99)

        # Fill missing values 先用每一列的均值填补缺失；如果仍然有没填完的（例如整列是 NaN），则填 0。
        X = X.fillna(X.mean()).fillna(0)

        # is_training = True：→ 有完整的训练数据（含目标变量），需要拟合（fit）标准化器，确定数据的统计特征（均值、分位数等）。
        # is_training = False：→ 预测阶段，使用前面“训练阶段”保存的标准化器参数进行转换（transform），保证缩放方式一致。
        # 否则，如果在测试数据上重新 fit，就会引入“未来信息”，造成数据泄露。
        if is_training:
            # 使用 RobustScaler 对所有特征进行鲁棒标准化（robust scaling）
            X_scaled = self.scaler.fit_transform(X)
            self.all_feature_names = list(X.columns)
        else:
            # 预测时，如果当前输入 X 缺少某些特征列，则在这里自动补上，默认值为 0
            for col in self.all_feature_names:
                if col not in X.columns:
                    X[col] = 0
            # 重新排列列顺序
            X = X[self.all_feature_names]
            # 应用已拟合的标准化器
            X_scaled = self.scaler.transform(X)
        # 将缩放后的 NumPy 数组重新包装成 DataFrame，保持结构信息
        return pd.DataFrame(X_scaled, columns=self.all_feature_names, index=df.index)

    def select_features(self, X_full):
        """
        根据当前配置执行特征选择：
        - 如果设置了 manual_features，则直接使用这些列；
        - 否则用 SelectKBest(mutual_info_regression) 自动筛选。
        """
        # self.manual_features='E19', 'I2', 'I4', 'P10', 'P11', 'P5', 'S8', 'V13', 'V7', 'V9'
        self.manual_features = 'E19','I4', 'P11','S8', 'V13','V9'
        

        # 手动模式
        if self.manual_features is not None:
            print(f"⚙️ Using {len(self.manual_features)} manually specified features")
            # 检查缺失列
            missing = [f for f in self.manual_features if f not in X_full.columns]
            if missing:
                print(f"⚠️ Warning: Missing features in data: {missing}")
            self.feature_names = [f for f in self.manual_features if f in X_full.columns]
            X_selected = X_full[self.feature_names].copy()

        # 自动模式
        else:
            print(f"🔍 Automatically selecting top {self.feature_selection_k} features...")
            X_selected = self.feature_selector.transform(X_full)
            selected_indices = self.feature_selector.get_support(indices=True)
            self.feature_names = [self.all_feature_names[i] for i in selected_indices]

        X_selected = pd.DataFrame(X_selected, columns=self.feature_names)

        print("Selected features:", self.feature_names)
        
        return X_selected

    def fit_from_file(self, train_path, target_col='forward_returns'):
        """Fit the model using fixed train/test split (8810 train, 180 test)"""
        print("Loading training data...")
        train_df = pd.read_csv(train_path)

        print("Preparing features...")
        X_full = self.prepare_features(train_df, is_training=True)

        # 计算最优仓位作为目标变量
        market_excess_returns = train_df['forward_returns'] - train_df['risk_free_rate']
        market_excess_cumulative = (1 + market_excess_returns).prod()
        market_mean_excess_return = (market_excess_cumulative) ** (1 / len(train_df)) - 1
        c = (1 + market_mean_excess_return) ** (1 / (market_excess_returns > 0).mean()) - 1
        train_df["target"] = (c / market_excess_returns).clip(0, 2)
        y = train_df["target"].fillna(0)

        print("best score (full data):", score(train_df, pd.DataFrame({'prediction': y}), ''))
        # ==========================================================

        split_point = 8000
        
        # ===== 调用特征选择函数 =====
        self.feature_selector.fit(X_full[:split_point], y[:split_point])
        X_final = self.select_features(X_full)

        
        X_train, X_test = X_final.iloc[:split_point], X_final.iloc[split_point:]
        y_train, y_test = y.iloc[:split_point], y.iloc[split_point:]
        # 获取测试集的solution数据并重置索引
        test_solution = train_df.iloc[split_point:].copy().reset_index(drop=True)

        results = {}
        for name, model in self.models.items():
            try:
                print(f"\nTraining {name} model...")
                model.fit(X_train, y_train)

                y_train_pred = model.predict(X_train).clip(0, 2)
                y_test_pred = model.predict(X_test).clip(0, 2)

                # 计算指标
                rmse_train = np.sqrt(np.mean((y_train_pred - y_train) ** 2))
                rmse_test = np.sqrt(np.mean((y_test_pred - y_test) ** 2))

                # Sharpe-like得分（用比赛自带score函数）
                score_train = score(
                    train_df.iloc[:split_point],
                    pd.DataFrame({'prediction': y_train_pred}),
                    ''
                )
                score_test = score(
                    test_solution,  # 使用已经重置索引的测试集solution
                    pd.DataFrame({'prediction': y_test_pred}),
                    ''
                )

                print(f"Results for {name}:")
                print(f"  RMSE_train={rmse_train:.6f}, RMSE_test={rmse_test:.6f}")
                print(f"  Sharpe_train={score_train:.6f}, Sharpe_test={score_test:.6f}")

            except Exception as e:
                print(f"Error training {name}: {e}")
                self.model_weights[name] = 0

        self.is_fitted = True

        return self

    def predict_batch(self, df_batch):
        if not self.is_fitted:
            raise ValueError("Model must be fitted before making predictions")

        X_full = self.prepare_features(df_batch, is_training=False)
        X_final = self.select_features(X_full)

        predictions = np.zeros(len(X_final))
        for name, model in self.models.items():
            # dict.get(key, default) default：如果键不存在，就返回这个默认值（而不是报错）
            if self.model_weights.get(name, 0) > 0:
                try:
                    pred = model.predict(X_final)
                    predictions += self.model_weights[name] * pred
                except Exception as e:
                    print(f"Error in prediction with {name}: {e}")

        return predictions[0] if len(predictions) == 1 else predictions



In [ ]:
def create_prediction_function():
    """Create prediction function that captures trained model"""
    pipeline = FixedSP500Pipeline(feature_selection_k=10)

    train_path = '/kaggle/input/hull-tactical-market-prediction/train.csv'
    if os.path.exists(train_path):
        pipeline.fit_from_file(train_path)
    else:
        print("Warning: No training data found")
        return lambda test: 0.0

    def predict(test: pl.DataFrame) -> float:
        try:
            test_df = test.to_pandas()
            prediction = pipeline.predict_batch(test_df)
            signal = np.clip(prediction, 0.0, 2.0)
            return float(signal)
        except Exception as e:
            print(f"Prediction error: {e}")
            return 1.0

    return predict


predict = create_prediction_function()

inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if __name__ == "__main__":
    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        inference_server.serve()
    else:
        inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))